## Evaluate Using a Custom Evaluator

Contoso Airways is developing a customer support chatbot to assist customers with managing their flight reservations, offering real-time help with booking changes, cancellations, and seat selection. The AI-powered chatbot is designed to provide quick, accurate responses, streamlining the travel experience and reducing wait times for customer service inquiries.

In this exercise, you will assess the friendliness of the customer support chatbot based on how the model responds during a customer interaction.

## Add environment variables to the .env file

In the root of the **Evaluation and Data Generation Workshop** folder is an `.env` file. Within the `.env` file, fill in the values for the environment variables. You can locate the values for each environment variable in the following locations of the [Azure AI Foundry](https://ai.azure.com) portal:

- `AZURE_SUBSCRIPTION_ID` - On the **Overview** page of your project within **Project details**.
- `AZURE_AI_PROJECT_NAME` - At the top of the **Overview** page for your project.
- `AZURE_OPENAI_RESOURCE_GROUP` - On the **Overview** page of the **Management Center** within **Project properties**.
- `AZURE_OPENAI_SERVICE` - On the **Overview** page of your project in the **Included capabilities** tab for **Azure OpenAI Service**.
- `AZURE_OPENAI_API_VERSION` - On the [API version lifecycle](https://learn.microsoft.com/azure/ai-services/openai/api-version-deprecation#latest-ga-api-release) webpage within the **Latest GA API release** section.
- `AZURE_OPENAI_ENDPOINT` - On the **Details** tab of your model deployment within **Endpoint** (i.e. **Target URI**)
- `AZURE_OPENAI_DEPLOYMENT_NAME` -  On the **Details** tab of your model deployment within **Deployment info**.

# Sign in to Azure

As a security best practice, we'll use [keyless authentication](https://learn.microsoft.com/azure/developer/ai/keyless-connections?tabs=csharp%2Cazure-cli) to authenticate to Azure OpenAI with Microsoft Entra ID. Before you can do so, you'll first need to install the **Azure CLI** per the [installation instructions](https://learn.microsoft.com/cli/azure/install-azure-cli) for your operating system.

Next, open a terminal and run `az login` to sign in to your Azure account.

## Install the package

We'll need to load the Prompty file using the `load_flow` function which is available in the `promptflow.client` package. We'll begin by installing the package.

In [1]:
%pip install promptflow

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Access the environment variables.

We'll import `os` and `load_dotenv` so that you can access the environment variables.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

## Setup keyless authentication

Rather than hardcode your **key**, we'll use a keyless connection with Azure OpenAI.

In [3]:
import azure.identity

credential = azure.identity.DefaultAzureCredential()
token_provider = azure.identity.get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

token = token_provider()

## Import promptflow

We'll import `load_flow` from `promptflow` so that we can later load a flow and consume the flow object like a function by providing key-value arguments.

In [4]:
from promptflow.client import load_flow

## Configure the model_config

The `model_config` will be passed into `load_flow` when we later load the friendliness prompty. Let's configure the `model_config` with the following:

- Azure OpenAI endpoint
- Azure OpenAI key
- Azure deployment

In [5]:
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": token,
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")
}

## Create variables for the evaluation data

Although we could assign the `query` and `response` strings within the call to the friendliness evaluator, we'll set them here in case you'd like to try different pairs. We've included an alternate response that could be tested by removing the `#` and commenting out the initial `response` variable. The alternate `response` provides the ability to test how the friendliness prompty scores different responses.

In [6]:
query = "I've been on hold for 30 minutes just to ask about my luggage! This is ridiculous. Where is my bag?"
response = "I apologize for the long wait time, that must have been frustrating. I understand you're concerned about your luggage. Let me help you locate it right away. Could you please provide your bag tag number or flight details so I can track it for you?"
# response = "Your bag is currently at the airport."

## Load the custom evaluator and create a call to the Prompty

You're now ready to load the `friendliness.prompty` file and create a call to the evaluator. Using `load_flow`, we'll load `friendliness.prompty` and pass in the `model` configuration.

Next, we'll create a variable to store the score of the evaluation. To evaluate the data, we'll pass the `query` and `response` into the custom evaluator call.

In [7]:
friendliness_eval = load_flow(source="friendliness.prompty", model={"configuration": model_config})
friendliness_score = friendliness_eval(
    query=query,
    response=response
)

## Evaluate for friendliness

You're now ready to run the evaluation! We'll pass the `friendliness_score` into a print statement to view the results.

In [8]:
print(friendliness_score)

{"score": 5, "explanation": "The response is very friendly, empathetic, and proactive in offering assistance to resolve the issue."}


## Delete resources

If you've finished exploring Azure AI Services, delete the Azure resource that you created during the workshop.

**Note**: You may be prompted to delete your deployed model(s) before deleting the resource group.